# SECTION 03 本番のプログラムを作る
- このセクションで学ぶこと
  - クラスを設計する流れ
  - コンストラクタの実装
  - メソッドの役割を決める
  - メインメソッドの実装
  - 指定ページから絶対URL⼀覧を得るメソッドの実装
  - 絶対URLを画像とHTMLに振り分けるメソッドの実装
  - 画像データを保存するメソッドの実装
  - メインプログラムの実⾏
  - コラム: プログラムのテスト

## クラスを設計する流れ
- 複雑なプログラムでは「関数(メソッド含む)が別の関数を呼び出す」を繰り返すことで実現する
- 関数間で全てのデータを引数で受け渡しするとカオスになるので、クラスを使って共有データはインスタンス変数として定義する(詳細な理由は5章を参照)
- クラス設計
  - プログラムが必要とする情報はコンストラクタで受け取る
  - メソッド間で共有されるデータはコンストラクタでインスタンス変数として定義する
  - 処理に応じてメソッドを分けて、中⼼となるメソッドが各機能を担当するメソッドを呼び出すシンプルな処理の流れを作る

## このアプリケーションの設計概要

## コンストラクタの実装
- 引数
  - save_dirpath: 画像保存場所
  - start_page: 最初にアクセスするURL
  - maximum_download: 集める画像の枚数
    
- インスタンス変数
  - self.save_dirpath: 引数より
  - self.crawl_url_list: 次にアクセスするURLのリスト
  - self.stocked_url: すでにアクセスしたURLのセット
  - self.maximum_download: 引数より
  - self.download_counter: 何枚保存したか数える

In [1]:
import os
import re
import urllib
import requests
class ImageCrawller:
    def __init__(self, save_dirpath, start_page, maximum_download):
        self.save_dirpath = save_dirpath
        self.crawl_url_list = [start_page]
        self.stocked_url = set()
        self.maximum_download = maximum_download
        self.download_counter = 0

## メソッドの役割を決める
- メソッドの分割について
  - ⼤きい: 処理が多すぎてメソッドの⾒通しが悪くなる
  - ⼩さい: メソッド間の呼び出しが多すぎて分かりにくい
  - 適切な粒度(分割する⼤きさ)が必要
- 本章で作成するメソッド
  - コンストラクタ(作成済み)
  - 下記3つを束ねる中⼼となるメソッド(メインメソッド)
  - あるURLのHTMLに含まれる絶対URLをリストで返すメソッド
  - URLをHTMLと画像に振り分けるメソッド
  - イメージを保存するメソッド

## メインメソッドの実装
- 荒削りでよいので、全体の処理の流れをコメント付きで書く
- 細部を実装してからメインメソッドも微修正される
- 実験的に主要な機能をテストしていないと設計しづらく、あとにプログラムの⼤きな修正が発⽣する可能性が⾼まる
- プログラム(次ページ)の設計
  1. 終了条件に合致していないか確認。合致していれば終了
  2. URL回収⽤のウェブページURLをリストから取得
  3. そのURLにアクセスして絶対URLのリストを取得
  4. 絶対URLのリストを「画像」「ウェブページ」に分類
  5. 画像のURLリストの画像を全て取得
  6. 1に戻る(whileによる無限ループ)

In [ ]:
def run(self):
    while True:
        # 処理1: 探索するURLがなければ終了。規定数以上を集めていても終了
        if len(self.crawl_url_list) == 0:
            break
        if self.download_counter >= self.maximum_download:
            break
        # 処理2: 次に調べるHTMLのURLを取得
        crawl_url = self.crawl_url_list.pop(0)
        # 処理3: HTMLページから絶対URLを抽出する
        urls = self.get_abs_urls(crawl_url)
        # 処理4: 絶対URLをHTMLかイメージかに分類する。イメージのリストを返す
        image_url_list = self.get_image_url_list(urls)
        # 処理5: リストに格納されたイメージを全て保存する
        self.save_images(image_url_list)
    print('Finished')

## 指定ページから絶対URL⼀覧を得るメソッドの実装

In [ ]:
def get_abs_urls(self, url):
    try:
        # URLから⽂字列のHTMLを取得
        response = requests.get(url)
        html = response.text
        # HTMLからURLを抜き出してリストに格納
        relative_url_list = re.findall('<a href="?\'?([^"\'>]*)', html)
        # 相対URLを絶対URLに変換。HTTP/HTTPS以外のURLは除外
        abs_url_list = []
        for relative_url in relative_url_list:
            abs_url = urllib.parse.urljoin(url, relative_url)
            if abs_url.startswith('http://') or abs_url.startswith('https://'):
                abs_url_list.append(abs_url)
        return abs_url_list
    except Exception as e:
        print('Error: {}'.format(e))
        return []

## 絶対URLを画像とHTMLに振り分けるメソッドの実装

In [ ]:
def get_image_url_list(self, url_list):
    try:
        image_url_list = []
        for url in url_list:
            if url in self.stocked_url: # すでに登録されたURLなので無視
                continue
            if '.jpg' in url:
                image_url_list.append(url)
            elif '.png' in url:
                image_url_list.append(url)
            elif '.gif' in url:
                image_url_list.append(url)
            else:
                self.crawl_url_list.append(url) # 画像ファイルではないのでURL取得に使う
        self.stocked_url.add(url) # URLを登録。同じものは再登録しない
        return image_url_list
    except Exception as e:
        print('Error: {}'.format(e))
        return []

## 画像データを保存するメソッドの実装

In [ ]:
def save_images(self, image_url_list):
    for image_url in image_url_list:
        try:
            # 決められた回数以上のダウンロードをした場合は終了
            if self.download_counter >= self.maximum_download:
                return
            # イメージを取得
            response = requests.get(image_url, stream=True)
            image = response.content
            # イメージをファイルに保存
            file_name = image_url.split('/').pop()
            save_path = os.path.join(self.save_dirpath, file_name)
            fout = open(save_path, 'wb')
            fout.write(image)
            fout.close()
            self.download_counter += 1
            print('saved image: {}/{}'
            .format(self.download_counter, self.maximum_download))
        except Exception as e:
            print('Error: {}'.format(e))

## メインのプログラムの実⾏
- プログラム最後に「if __name__ == ' __main__ ' :」を定義

In [ ]:
if __name__ == '__main__':
    save_dirpath = 'test'
    start_page = 'https://gihyo.jp/book/list'
    maximum_download = 10
    crawller = ImageCrawller(save_dirpath, start_page, maximum_download)
    crawller.run()

## コラム: プログラムのテスト
- 数⼗⾏以上のコードがミスなく書けていると思わないこと
- 「ビッグバンテスト」と呼ばれる全てを書き終わってから動かしてテストをすることは避けること
- テストを使ってきちんと動くプログラムを書くコツ
  - モジュールの設計で役割ごとにプログラムファイルを分離
  - 確実に動く⼩さなプログラムを開発してモジュールに組み込む
  - モジュール間の依存関係を減らしてモジュールレベルでテストを実施できるようにする
  - 変更を繰り返した汚いコードは綺麗に書き直す
  - ユニットテストでテストを⾒える化する(中級者)
  - ⾃動化などでテストを勝⼿に⾛らせるようにする(上級者)

## 演習
- 最初にアクセスするページを変更してアプリケーションを⾃分で起動してみる(ウェブサイトに過負荷を与えるのを避けるため⼩規模のサイトは使わないでください)
- 画像ではなくHTMLを集めて保存するプログラムを作成してください
- 発展課題: HTMLがすでに保存されているファイル名であれば名前に連番を付けるなどしてください